In [71]:
import functools

import os
import time

In [72]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver import Keys

# from selenium.webdriver.chrome.service import service as ChromeService

In [73]:
BROWSER_EXECUTABLE_PATH = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"

In [74]:
PERPLEXITY_URL = "https://www.perplexity.ai/"

In [75]:
# A decorator that immediately attempts a block of code up to an optionally specified number of times.
def attempt_immediately(max_num_attempts=5, pause_between_attempt_time=0) -> bool:
    '''
        This decorator attempts to run code (func) for a specified number of times (max_num_attempts).
        It will pause for a specified number of seconds (pause_between_attempt_time) in between attempts.
    '''

    # When the "attempt_immediately" function is called, it returns the "decorator_attempt_immediately" function
    # (with the appropriate arguments) that the decorator is actually applied to.

    def decorator_attempt_immediately(func):

        # Provide a standard decorator wrapper to be able to use functools to preserve information
        # about the original function.
        @functools.wraps(func)
        def wrapper_attempt_immediately(func):
            current_num_attempts = 0
            
            while current_num_attempts < max_num_attempts:
                try:
                    func()
                    # If the code runs successfully, no further actions is required.
                    return True
                except:
                    # If the code fails 
                    time.sleep(pause_between_attempt_time)
                    current_num_attempts += 1
            
            # Exit if the code failed to run.
            return False

        return wrapper_attempt_immediately(func)
    
    # Immediately execute the function and return the result.
    return decorator_attempt_immediately

In [80]:
# Use Brave Browser as the Chrome-based browser for this program.
options = webdriver.ChromeOptions()
options.binary_location = BROWSER_EXECUTABLE_PATH

# Options to hide use
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Create a Desktop Webdriver
driver = webdriver.Chrome(
    # service=service,
    options=options
)

# Options to Hide Driver
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

In [81]:

driver.get("https://google.com")
# driver.get("https://shopify.com")
# driver.get(PERPLEXITY_URL)

# # Login to Perplexity

# @attempt_immediately()
# def login():

#     time.sleep(30)

#     # Wait for CloudFlare protection to be acknowledged.
#     @attempt_immediately(max_num_attempts=20, pause_between_attempt_time=5)
#     def click_login_button():

#         # Locate the "Sign in" button from the list of elements with the "leading-loose" class.
#         leading_loose_elements = driver.find_elements(By.CLASS_NAME, "leading-loose")

#         for element in leading_loose_elements:
#             if element.text == "Sign in":
#                 element.click()
#                 break

#     driver.find_element()


#     @attempt_immediately()
#     def send_email():
#         # Locate the email form from the list of elements of the "input" type.
#         elements = driver.find_elements(By.TAG_NAME, "input")

#         elements[2].send_keys(os.getenv("PERPLEXITY_EMAIL"))

#     # Click on the login button.
#     driver.find_elements(By.TAG_NAME, "button")[20].click()

#     # Click on the the manual code button
#     driver.find_element(By.CLASS_NAME, "text-align-center")

In [112]:
# @attempt_immediately(pause_between_attempt_time=3)
def preform_request(query, image_path=None):

    driver.get(PERPLEXITY_URL)

    # Text Box
    driver.find_element(By.CLASS_NAME, "caret-superDuper").send_keys(query)

    if image_path != None:
        driver.find_element(By.TAG_NAME, "input").send_keys(image_path)

    # Click Submit Button
    driver.find_elements(By.TAG_NAME, "button")[7].click()

    time.sleep(3)

    # Wait for the generation to finish by watching for the stop icon to disappear
    @attempt_immediately()
    def locate_plus_button():
        while True:
            try:
                driver.find_element(By.CLASS_NAME, "fa-circle-stop")
                time.sleep(1)
            except:
                print("Exiting!")
                return

    # Locate text content
    text_elements = None
    @attempt_immediately()
    def locate_completed_text():
        nonlocal text_elements
        text_block = driver.find_element(By.CLASS_NAME, "prose")
        text_elements = text_block.find_elements(By.TAG_NAME, "span")

        assert len(text_elements) > 2

    text = ""
    for element in text_elements:
        text += " " + element.text

    return text

In [83]:
# a = driver.find_element(By.CLASS_NAME, "caret-superDuper")
# # b = a.find_element(By.TAG_NAME, "textarea")

In [84]:
# a.send_keys("ABC")

In [99]:
# b.text

In [113]:
preform_request("What time is it?")

Exiting!


'The current time is 04:15 AM (PDT) on Sunday, October 13, 2024.'

In [114]:
preform_request("Why does Calculus work?")

Exiting!


'Calculus works by providing a mathematical framework for understanding and analyzing continuous change. It does this through two main branches: differential calculus and integral calculus.Differential calculus focuses on the study of rates of change and slopes of curves. It helps in understanding how functions change as their input changes. This is achieved through the concept of derivatives, which measure the rate of change of a function at a given point224\n.4\n.4Integral calculus, on the other hand, deals with the accumulation of quantities. It is used to find areas under curves, volumes of solids, and other quantities that can be represented as the sum of many small parts. This is done through the process of integration, which is essentially the reverse of differentiation224\n.4\n.4The fundamental theorem of calculus establishes a deep connection between differential and integral calculus. It shows that differentiation and integration are inverse processes, meaning that the deriva